<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Gestion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Gestión de datos: Join, Combine, y Reshape](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=37Vujga1JOOK)

>>[Indice Jerárquico](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=g-QfwPOyJ0JO)

>>[Reordenando los diferentes niveles](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=fW0-QPnwM2X2)

>>[Indexing columnas en un Dataframe](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=5nEdvsjbPN9y)

>[Combinando datasets](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=ONyrRGhIQQaE)

>>[Database-Style joins en Dataframes](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=4aHveRwpQkdS)

>>>[Argumentos de la función Merge](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=S3dWfNjVkOfp)

>>[Merge en el índice](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=SnS0Dvcs0pFL)

>>[Concatenando entre ejes](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=DI3tPxyq4z_l)

>>[Reshaping y Pivot](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=VqNU7hfT7B75)

>>>[Pivotar el formato "largo" a "ancho"](#updateTitle=true&folderId=1hYY6URNFLa2w5I3uQbpDlwOox_am-5cM&scrollTo=AKcrY6W3ADtt)



# Gestión de datos: Join, Combine, y Reshape

En muchas aplicaciones, los datos son distrbuidos a través de un diferentes archivos y base de datos, o en un formato que no es fácil de analizar. 
Para ello, utilizaremos herramientas que nos facilitarán el proceso de preparación de los datos. 

## Indice Jerárquico

Los índices jerárquicos nos permiten tener múltiples índices en un mismo eje.

Esto nos permitiría visualizar datos de una dimensión superior en una inferior (Ejemplo: ventas indexadas primero por local y dentro de cada local, a través del tiempo)

Esta es una herramienta de Pandas que ya vimos anteriormente, pero ahora vamos a dar ejemplos diferentes.

In [ ]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

Lo que está viendo es una vista predefinida de una Serie con un MultiIndex como índice. Los espacios en la visualización del índice significan "usar la etiqueta directamente arriba".
Con este tipo de índices puedes realizar lo que se llama partial-index, lo que permite obtener un subset de los datos.

In [ ]:
data['b']

In [ ]:
data.loc[:, 2] # La selección es posible incluso desde dentro del nivel
# En este caso se hace "loc" que es filtrado por nombre de indice, pero como 
# el indice tiene de nombre "2", se queda con esa clave.

Los índices jerárquicos juegan un rol importante en el modelado y agrupamiento de datos. Por ejemplo, podemos reordenar los datos anteriores en un Dataframe usando el método **unstack**.

Desapilar podría introducir datos faltantes si no se encuentran todos los valores en el nivel en cada uno de los subgrupos:

In [ ]:
obj_desapilado = data.unstack()
obj_desapilado

In [ ]:
obj_apilado = obj_desapilado.stack() # Es la función inversa
obj_apilado

## Reordenando los diferentes niveles
Con los Dataframe, los ejes pueden tener índice jerárquicos también. Veamos el siguiente ejemplo:

In [ ]:
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Marty', 'Marty', 'Doc'],
                              ['Lorraine', 'George', 'Delorean']])
df

In [ ]:
df.index.names = ['key1', 'key2'] # Los niveles pueden tener nombres. 
df

In [ ]:
df['Marty'] # Podemos seleccionar datos parciales

In [ ]:
df.swaplevel('key1','key2') # También es posible reordenar por niveles

Finalmente, podemos aplicar funciones estadísticas en Dataframes o series, como agregación en un eje particular. 
Considerando el ejemplo anterior, podemos realizar una agregación por nivel, ya sea por fila o columna: 

In [ ]:
df.sum(level='key2')

## Indexing columnas en un Dataframe
Es muy común que algunas veces desees mover algunos índices de columnas a filas, o viceversa. Para ello podemos utilizar la función **set_index**

In [ ]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
df

In [ ]:
df2 = df.set_index(['c', 'd'])
df2

Si agregamos el parámetro drop = False, las columnas c y d pasarán a ser índices de fila, pero también se mantendrán las columnas originales con sus valores correspondientes:

In [ ]:
df3 = df.set_index(['c', 'd'], drop = False)
df3



---


# Combinando datasets

Los datos contenidos en los objetos Pandas se pueden combinar de varias maneras:



*   **pandas.merge** conecta filas en DataFrames en función de una o más claves. Esto será familiar para los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de unión de bases de datos.
*   **pandas.concat** concatena o "apila" objetos juntos a lo largo de un eje.
*   **combine_first** permite unir datos superpuestos para completar los valores faltantes en un objeto con valores de otro.








## Database-Style joins en Dataframes

Las operaciones de **merge** o **join** combinan conjuntos de datos al vincular filas con una o más claves. Estas operaciones son centrales para las bases de datos relacionales.

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('\n')
print(df2)

Este es un ejemplo de una unión de muchos a uno; los datos en df1 tienen varias filas etiquetadas con a y b, mientras que df2 tiene solo una fila para cada valor en la columna clave. Llamando a fusionar con estos objetos obtenemos:  

In [ ]:
pd.merge(df1,df2)

Puedes observar que no especifiqué en qué columna unir. Si no se especifica esa información, la combinación usa los nombres de columna superpuestos (es decir, los nombres en comun que tengan ambos dataframes) como las claves. Sin embargo, es una buena práctica especificar explícitamente:

In [ ]:
pd.merge(df1, df2, on='key')

Como en las sentencias SQL, si los nombres de las columnas son diferentes en cada objeto, se puede especificar de forma separada.

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print (df3)
print('\n')
print(df4)

In [ ]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

Puedes notar que los valores **c** y **d** y los datos asociados faltan en el resultado. Por defecto, merge hace una unión 'interna' (inner join); Las claves en el resultado son la intersección, o el conjunto común que se encuentra en ambas tablas. Otras opciones posibles son **izquierda**, **derecha** y **exterior**. La unión externa toma la unión de las claves, combinando el efecto de aplicar las uniones izquierda y derecha.
A continuación encontrarás un diagrama con de SQL joins con su correspondiente sentencia en ese lenguaje que te será de utilidad para el desarrollo de tus actividades

<img src = "https://i.pinimg.com/564x/42/48/72/424872ac0b25c05e117b521d55616551.jpg">


A continuación se detallan las opciones que se encuentran disponibles en Pandas con el compartamiento asociado

Opción | Comportamiento
-------|-------
**inner**| Utiliza solo la combinación de claves comunes para ambas tablas
**left** | Utiliza solo la combinación de claves encontradas en la tabla declarada a izquierda
**right** | Utiliza solo la combinación de claves encontradas en la tabla declarada a derecha
**outer** | Utiliza solo la combinación de claves observada en ambas tablas juntas 

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('\n')
print(df2)

In [ ]:
pd.merge(df1, df2, how='left')

Para realizar merge con multiples keys, debemos pasar una lista de nombres de columnas

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})


In [ ]:
print (left)
print ('\n')
print(right)

In [ ]:
pd.merge(left, right, on=['key1', 'key2'], how='inner')

###Argumentos de la función **Merge**

A continuación se detallan los argumentos más utilizados con la función **merge** asociados a su descripción.

Argumento | Descripción
---------|------------
left | Dataframe se fusiona en el lado izquierdo
right | Data frame se fusiona en el lado derecho
how | con parámetro 'inner', 'outer', 'left', o 'right'. Default 'inner'
on | La unión se hace en base a nombre de columnas. Deben estar presentes en ambos Dataframes
left_on| Se utilizan las columnas del Dataframe izquierdo como claves
right_on| Análogo al 'left_on'
left_index|Utiliza  el índice de fila en la izquierda como clave del join.
right_index|Análogo al 'left_index'
sort | Ordena datos fusionados lexicográficamente por las claves.
suffixes|Tupla de valores de cadenas a agregar a una coluna en caso de sobreposición. Si por ejemplo tenemos data en ambos dataframes podemos agregar data_x, data_y como sufijos
copy|Si es falso, evita copiar datos en la estructura resultante.
indicator|Agrega una columna especial '_merge' que indica la fuente de cada fila. Valores pueden ser 'left_only', 'right_only', o, 'both'.




## Merge en el índice

En algunos casos, la clave del merge se dará en el índice. En este caso, puedes pasar el parámetro **left_index=true** o **right_index=true** (o ambos) para indicar que el índice será usado como clave de merge. 


In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
              'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [ ]:
print (left1)
print ('\n')
print (right1)

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how = 'inner') # Qué sucede aquí?

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # y aquí?

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
            index=['a', 'c', 'e'],
            columns=['Devoto', 'Palermo'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
            index=['b', 'c', 'd', 'e'],
            columns=['Belgrano', 'Colegiales'])

otro_df= pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
            index=['a', 'c', 'e', 'f'],
            columns=['Villa Urquiza', 'Nuñez'])

In [ ]:
left2

In [ ]:
right2

In [ ]:
otro_df

Para merge simples de índice sobre índice, puede pasar una lista de DataFrames para unirse como alternativa al uso de la función concat más general que se describe en la siguiente sección.

En este caso, los índices del dataframe al que le aplico el join son los que se mantendrán:

In [ ]:
left2.join([right2, otro_df])

## Concatenando entre ejes
Otro tipo de operación de combinación de datos se conoce indistintamente como concatenación, enlace o apilamiento. La función concatenada de NumPy puede hacer esto con las matrices NumPy

In [ ]:
arreglo = np.arange(12).reshape((3,4))
arreglo

In [ ]:
np.concatenate([arreglo, arreglo], axis=1)

En el contexto de objetos pandas como Series y DataFrame, tener ejes etiquetados le permite generalizar aún más la concatenación de matriz

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
pd.concat([s1, s2, s3])

Por defecto, concat funciona a lo largo de axis = 0, produciendo otra serie. Si pasa axis = 1, el resultado será un DataFrame (axis = 1 son las columnas)

In [ ]:
pd.concat([s1, s2, s3], axis=1) #Observar que tras correr este concat, las "columnas" hacen referencia al dataframe de origen de cada dato

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['s1', 's2', 's3'])

## Reshaping y Pivot

Existen varias operaciones básicas para reorganizar datos tabulares. Estos se denominan alternativamente operaciones de Reshaping y Pivot.

La indexación jerárquica proporciona una forma consistente de reorganizar los datos en un DataFrame. Hay dos acciones principales:

*   **Apilar**: Esto "gira" las columnas en los datos a las filas
*   **Desapilar**: Esto gira de las filas a las columnas

Ilustraremos estas operaciones a través de una serie de ejemplos. Considere un pequeño DataFrame con matrices de cadenas como índices de fila y columna:

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='Estado'),
                    columns=pd.Index(['Uno', 'Dos', 'Tres'], name='Número'))
data

In [ ]:
resultado = data.stack()
resultado

Hasta ahora, este ejemplo es similar al que vimos al principio de la clase, ahora vayamos más allá. 


In [ ]:
df = pd.DataFrame({'left': resultado, 'right': resultado + 5},
                  columns=pd.Index(['left', 'right'], name='Lado'))
df

**¿Qué sucede si apilamos/desapilamos sobre un índice?**

Cuando se desapila en un DataFrame, el nivel desapilado se convierte en el nivel más bajo en el resultado:

In [ ]:
df.unstack('Estado')

In [ ]:
df.unstack('Estado').stack('Lado')

In [ ]:
df_new = df.unstack('Estado').stack('Lado')
df_new

In [ ]:
df_colorado = df_new[['Colorado']]
df_colorado

### Pivotar el formato "largo" a "ancho"

Una forma común de almacenar múltiples series de tiempo en bases de datos y CSV es en el llamado formato largo o apilado. Carguemos algunos datos de ejemplo y hagamos una pequeña cantidad de disputas de series de tiempo y otra limpieza de datos:

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/macrodata.csv')
data.head(5)

In [ ]:
periodos = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='fecha') #El método PeriodIndex combina las columnas de año y trimestre
                                                                              # para crear un tipo de intervalo de tiempo.

columnas = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columnas) # reindex le cambia el indice a un Dataframe                                                                            

data.index = periodos.to_timestamp(freq='D', how = 'end') # Castea los valores "year-quarter" que definimos arriba a tipo DateTimeIndex
# freq = 'D' is "default"
# Si el parametro how = "start" --> Para el year: 1959, quarter: 01 asignará el valor "1959-01-01"
# Si el parametro how= "end" --> Asignará el valor "1959-03-31"

ldata = data.stack().reset_index().rename(columns={0: 'valor'})

In [ ]:
ldata[:10]

Bueno, esto está medio raro. ¿En verdad la hora es un dato relevante?

In [ ]:
import datetime as dt
ldata['fecha'] = ldata['fecha'].dt.strftime('%Y-%m-%d')
ldata.head(10)

Este es el llamado formato largo para múltiples series de tiempo u otros datos de observación con dos o más claves (aquí, nuestras claves son fecha y elemento). Cada fila de la tabla representa una sola observación.

Los datos se almacenan con frecuencia de esta manera en bases de datos relacionales como MySQL, ya que un esquema fijo (nombres de columna y tipos de datos) permite que el número de valores distintos en la columna del elemento cambie a medida que se agregan datos a la tabla. 

En el ejemplo anterior, la fecha y el elemento generalmente serían las claves principales (en el lenguaje de la base de datos relacional), ofreciendo integridad relacional y uniones más fáciles. En algunos casos, los datos pueden ser más difíciles de trabajar en este formato; es posible que prefieras tener un DataFrame que contenga una columna por valor de elemento distinto indexado por marcas de tiempo en la columna de fecha. El método pivote de DataFrame realiza exactamente esta transformación:

In [ ]:
pivoted = ldata.pivot('fecha', 'item', 'valor') # Los primeros dos valores son las columnas que se utilizarán 
# respectivamente como el índice de fila y columna, luego, finalmente, una columna de valor opcional para llenar el DataFrame.
pivoted

 Suponga que tiene dos columnas de valor que desea remodelar simultáneamente:

In [ ]:
ldata['valor2'] = np.random.randn(len(ldata))
ldata[:10]

Omitiendo el último argumento, obtienes un DataFrame con columnas jerárquicas

In [ ]:
pivoted = ldata.pivot('fecha', 'item')
pivoted[:5]

In [ ]:
pivoted['valor'][:5]

Pivot es equivalente a crear un índice jerárquico usando **set_index** seguido de una llamada para desapilar. Claramente, es preferible utilizar pivot en su lugar:

In [ ]:
unstacked = ldata.set_index(['fecha', 'item']).unstack('item')
unstacked[:7]

### Pivotar el formato "ancho" a "largo"

Una operación inversa para pivotar en DataFrames es **pandas.melt**. En lugar de transformar una columna en muchas en un nuevo DataFrame, combina varias columnas en una, produciendo un DataFrame que es más largo que la entrada.

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [ ]:
union = pd.melt(df, ['key'])
union

In [ ]:
'''
Usando pivot, podemos volver a dar forma al diseño original
'''
reshaped = union.pivot('key', 'variable', 'value')
reshaped

Dado que el resultado de pivot crea un índice a partir de la columna utilizada como etiquetas de fila, es posible que queramos usar reset_index para mover los datos nuevamente a una columna

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])